<a href="https://colab.research.google.com/github/ShotaroBaba/NLP_Practice/blob/NLP_smaller_dataset/NLPIMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Firstly, download text dataset.:
import os
import gensim
import requests 
from gensim.parsing.preprocessing import stem
from gensim.parsing.preprocessing import strip_punctuation
from gensim.corpora import Dictionary
urlToWiki = "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
savedFileName = 'aclImdb_v1.tar.gz'
if not os.path.isfile(savedFileName):
  with open(savedFileName,'wb') as output:
    output.write(requests.get(urlToWiki).content)

In [0]:
import tarfile

# Extract tar.gz file
tarfile.open(savedFileName, "r:gz").extractall()

In [0]:
!cd aclImdb/

In [8]:
!ls aclImdb/test

labeledBow.feat  neg  pos  urls_neg.txt  urls_pos.txt


In [10]:
!cat aclImdb/test/urls_pos.txt

http://www.imdb.com/title/tt0406816/usercomments
http://www.imdb.com/title/tt0406816/usercomments
http://www.imdb.com/title/tt0406816/usercomments
http://www.imdb.com/title/tt0406816/usercomments
http://www.imdb.com/title/tt0406816/usercomments
http://www.imdb.com/title/tt0406816/usercomments
http://www.imdb.com/title/tt0406816/usercomments
http://www.imdb.com/title/tt0406816/usercomments
http://www.imdb.com/title/tt0406816/usercomments
http://www.imdb.com/title/tt0406816/usercomments
http://www.imdb.com/title/tt0406816/usercomments
http://www.imdb.com/title/tt0406816/usercomments
http://www.imdb.com/title/tt0406816/usercomments
http://www.imdb.com/title/tt0406816/usercomments
http://www.imdb.com/title/tt0406816/usercomments
http://www.imdb.com/title/tt0406816/usercomments
http://www.imdb.com/title/tt0406816/usercomments
http://www.imdb.com/title/tt0085461/usercomments
http://www.imdb.com/title/tt0085461/usercomments
http://www.imdb.com/title/tt0085461/usercomments
http://www.imdb.com/

In [0]:


# References
# Andrew L. Maas, Raymond E. Daly, Peter T. Pham, Dan Huang, Andrew Y. Ng, and Christopher Potts. (2011). Learning Word Vectors for Sentiment Analysis. 
# The 49th Annual Meeting of the Association for Computational Linguistics (ACL 2011).